In [ ]:
import ffmpeg
import numpy as np
import cv2
from mtcnn.mtcnn import MTCNN
from glob import glob
from PIL import Image

In [ ]:
input_height = 480
input_width = 640
output_height = 224
output_width = 224

In [ ]:
detector = MTCNN()

In [ ]:
def detect(video):
    for img in video:
        for detected in detector.detect_faces(img):
            try:
                x0 = detected['box'][0]
                y0 = detected['box'][1]
                x1 = x0 + detected['box'][2]
                y1 = y0 + detected['box'][3]
                croped = img[y0:y1, x0:x1]
                resized = np.array(Image.fromarray(croped).resize((output_height, output_width)))
                yield resized
            except:
                continue

In [ ]:
def video_to_frames(path):
    out, _ = (
        ffmpeg
        .input(path)
        .output('pipe:', format='rawvideo', pix_fmt='rgb24')
        .run(capture_stdout=True)
    )
    frames = (
        np
        .frombuffer(out, np.uint8)
        .reshape([-1, input_height, input_width, 3])
    )
    return frames

In [ ]:
def process(files):
    for file in files:
        for face in detect(video_to_frames(file)):
            yield face.flatten()

In [ ]:
live = glob("data/FRAUD2-Sunlight/Live_Faces/*/*/*.avi")
paper = glob("data/FRAUD2-Sunlight/Paper_Faces/*/*/*.avi")

In [ ]:
%%time
live_processed = np.vstack(process(live))
np.save("live/sunlight.npy", live_processed)

In [ ]:
%%time
paper_processed = np.vstack(process(paper))
np.save("paper/sunlight.npy", paper_processed)

In [ ]:
types = ["Office", "Cloud", "Dark", "Natural"]
for t in types:
    print(t)
    live = glob("data/FRAUD2-{}/Live_Faces/*/*/*.avi".format(t))
    paper = glob("data/FRAUD2-{}/Paper_Faces/*/*/*.avi".format(t))
    live_processed = np.vstack(process(live))
    np.save("live/{}.npy".format(t), live_processed)
    paper_processed = np.vstack(process(paper))
    np.save("paper/{}.npy".format(t), paper_processed)

In [ ]:
types = ["Shade"]
for t in types:
    print(t)
    live = glob("data/FRAUD2-{}/Live_Faces/*/*/*.avi".format(t))
    paper = glob("data/FRAUD2-{}/Paper_Faces/*/*/*.avi".format(t))
    live_processed = np.vstack(process(live))
    np.save("live/{}.npy".format(t), live_processed)
    paper_processed = np.vstack(process(paper))
    np.save("paper/{}.npy".format(t), paper_processed)

# Combine and sampling

In [ ]:
arrays = []
for p in glob("data/live/*"):
    arrays.append(np.load(p))
stacked = np.vstack(arrays)
np.random.shuffle(stacked)
for i, a in enumerate(np.array_split(stacked, 128)):
    np.save("live/{}.npy".format(i), a)

In [ ]:
arrays = []
for p in glob("data/paper/*"):
    arrays.append(np.load(p))
stacked = np.vstack(arrays)
np.random.shuffle(stacked)
for i, a in enumerate(np.array_split(stacked, 128)):
    np.save("paper/{}.npy".format(i), a)